# 量子デバイス（QPU）の特性の違いを理解する
それぞれのデバイスでは、使えるゲートが異なります。IonQ, Rigetti については、前述のCirc() オペレータによる
量子コンピュータの１ビット２ビット量子操作が可能です。

## SV1 でできる回路演算について
サポートしている演算（ユニタリーオペレータ）を表示します

In [1]:
from braket.circuits import Gate
from braket.aws import AwsDevice
import string

# print all (the usual suspects) available gates currently available within SDK
gate_set = [attr for attr in dir(Gate) if attr[0] in string.ascii_uppercase]
print('Gate set supported by SDK:\n', gate_set)
print('\n') 

Gate set supported by SDK:
 ['CCNot', 'CNot', 'CPhaseShift', 'CPhaseShift00', 'CPhaseShift01', 'CPhaseShift10', 'CSwap', 'CY', 'CZ', 'H', 'I', 'ISwap', 'PSwap', 'PhaseShift', 'Rx', 'Ry', 'Rz', 'S', 'Si', 'Swap', 'T', 'Ti', 'Unitary', 'V', 'Vi', 'X', 'XX', 'XY', 'Y', 'YY', 'Z', 'ZZ']




---
## IonQ について
IonQ は、イオントラップという原理で作られた11量子ビットで構成されるデバイスです。

デバイスタイプとして、Rigetti の ARN `arn:aws:braket:::device/qpu/ionq/ionQdevice` を指定し、このデバイスで利用できる量子ゲートのタイプと、量子ビット間の結合（トポロジー）について調べてみます。

In [2]:
# import the device module
from braket.aws import AwsDevice

device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")

# get device name
device_name = device.name
# show supportedQuantumOperations (supported gates for a device)
device_operations = device.properties.dict()['action']['braket.ir.jaqcd.program']['supportedOperations']
print('Quantum Gates supported by {}:\n {}'.format(device_name, device_operations))

Quantum Gates supported by IonQ Device:
 ['x', 'y', 'z', 'rx', 'ry', 'rz', 'h', 'cnot', 's', 'si', 't', 'ti', 'v', 'vi', 'xx', 'yy', 'zz', 'swap', 'i']


In [3]:
# take a look at the device connectivity graph
device.properties.dict()['paradigm']['connectivity']

{'fullyConnected': True, 'connectivityGraph': {}}

IonQ は、11量子ビットがのそれぞれの量子ビットがそれ以外の全ての量子ビットと結合している、いわゆる全結合のトポロジーとなっています。そのため、任意の２量子ビットに対して、それぞれのデバイスに対してゲート操作が可能です。

---
## Rigetti について
Rigetti は、37量子ビットの超伝導デバイスで構成されています。

デバイスタイプとして、Rigetti の ARN `arn:aws:braket:::device/qpu/rigetti/Aspen-8` を指定します。

In [4]:
device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-8")

# get device name
device_name = device.name
# show supportedQuantumOperations (supported gates for a device)
device_operations = device.properties.dict()['action']['braket.ir.jaqcd.program']['supportedOperations']
print('Quantum Gates supported by {}:\n {}'.format(device.name, device_operations))

Quantum Gates supported by Aspen-8:
 ['cz', 'xy', 'ccnot', 'cnot', 'cphaseshift', 'cphaseshift00', 'cphaseshift01', 'cphaseshift10', 'cswap', 'h', 'i', 'iswap', 'phaseshift', 'pswap', 'rx', 'ry', 'rz', 's', 'si', 'swap', 't', 'ti', 'x', 'y', 'z']


In [5]:
# take a look at the device connectivity graph
device.properties.dict()['paradigm']['connectivity']

{'fullyConnected': False,
 'connectivityGraph': {'0': ['7'],
  '1': ['2', '16'],
  '2': ['1', '3', '15'],
  '3': ['2', '4'],
  '4': ['3', '5'],
  '5': ['4', '6'],
  '6': ['5', '7'],
  '7': ['0', '6'],
  '10': ['11', '17'],
  '11': ['10', '12', '26'],
  '12': ['11', '13', '25'],
  '13': ['12'],
  '15': ['2', '16'],
  '16': ['1', '15', '17'],
  '17': ['10', '16'],
  '20': ['21', '27'],
  '21': ['20', '22', '36'],
  '22': ['21', '23', '35'],
  '23': ['22', '24'],
  '24': ['23', '25'],
  '25': ['12', '24', '26'],
  '26': ['11', '25', '27'],
  '27': ['20', '26'],
  '30': ['31', '37'],
  '31': ['30', '32'],
  '32': ['31', '33'],
  '33': ['32', '34'],
  '34': ['33', '35'],
  '35': ['22', '34', '36'],
  '36': ['21', '35', '37'],
  '37': ['30', '36']}}

上記のように、Rigetti は 37量子ビットがそれぞれ、２量子ビット、または３量子ビットに対して結合していることがわかります。この結合規則を見ながら、操作可能な量子ゲートをプログラミングして量子計算を実施する必要があります。

---

## D-Waveについて

量子アニーラは、組合せ最適化の問題、特に2次制約なし最適化（QUBO）のクラスに属する問題を解決するために設計された特殊用途の量子コンピューターです。 Amazon Braketを使用すると、Braket-Oceanプラグインを介してD-WaveのOceanソフトウェアを使用してD-WaveQPUをネイティブにプログラムできます。

Amazon Braket ノートブックインスタンスには、Ocean と Braket-Ocean プラグインがプリインストールされています。

### D-Wave 2000Q
ARN `'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6'` を指定します。

In [6]:
device = AwsDevice('arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')

# take a look at the device connectivity graph
topology = device.properties.dict()['provider']['topology']
qubit_number = device.properties.dict()['provider']['qubitCount']
print(topology)
print('qubit number = ', qubit_number)

{'type': 'chimera', 'shape': [16, 16, 4]}
qubit number =  2048


### D-Wave the Advantage
ARN `arn:aws:braket:::device/qpu/d-wave/Advantage_system1` を指定します。

In [7]:
device = AwsDevice('arn:aws:braket:::device/qpu/d-wave/Advantage_system1')

topology = device.properties.dict()['provider']['topology']
qubit_number = device.properties.dict()['provider']['qubitCount']
print(topology)
print('qubit number = ', qubit_number)

{'type': 'pegasus', 'shape': [16]}
qubit number =  5760


---

これで、QPUのそれぞれの特性を知ることができました。それぞれのQPUのデバイス特性を意識しながら、量子計算のコードを書いていきましょう。